In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 47.1 MB/s eta 0:00:00

In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# Youtube Agent

In [ ]:
import requests

@tool
def search_youtube_video(query: str, region: str = "US", language: str = "en") -> dict:
    """
    Search for a video on YouTube using the RapidAPI YouTube138 API.

    Args:
        query (str): The search query (e.g., video title or keyword).
        region (str): The region code (default is 'US').
        language (str): The language code (default is 'en').

    Returns:
        dict: JSON containing details of the top video result.
    """
    url = "https://youtube138.p.rapidapi.com/search/"
    params = {
        "q": query,
        "hl": language,
        "gl": region
    }
    headers = {
        "x-rapidapi-key": "43e9e371cfmshbc3fc2a1afeaf39p134a77jsna5e15b538483",  # Replace with your RapidAPI key
        "x-rapidapi-host": "youtube138.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        for result in data.get("contents", []):
            video = result.get("video")
            if video:
                video_id = video.get("videoId")
                return {
                    "query": query,
                    "video_id": video_id,
                    "video_url": f"https://www.youtube.com/watch?v={video_id}",
                    "title": video.get("title"),
                    "description": video.get("descriptionSnippet"),
                    "duration_seconds": video.get("lengthSeconds"),
                    "published_time": video.get("publishedTimeText"),
                    "views": video.get("stats", {}).get("views"),
                    "thumbnail": video.get("thumbnails", [{}])[0].get("url"),
                    "channel": {
                        "title": video.get("author", {}).get("title"),
                        "channel_id": video.get("author", {}).get("channelId"),
                        "avatar": video.get("author", {}).get("avatar", [{}])[0].get("url"),
                        "channel_url": f"https://www.youtube.com/channel/{video.get('author', {}).get('channelId')}"
                    }
                }

        return {"message": "No video found for your query."}

    except requests.exceptions.RequestException as e:
        return {"error": f"Error during YouTube search: {str(e)}"}
    except (KeyError, IndexError, TypeError) as e:
        return {"error": f"Unexpected response format: {str(e)}"}

In [ ]:
youtube_assistant = create_react_agent(
    model=model,
    tools=[search_youtube_video],
    prompt=(
        "You are a helpful YouTube video assistant. Your job is to help users find YouTube videos "
        "based on their search queries using the `search_youtube_video` tool.\n\n"
        "🧠 Think step-by-step and always call the tool using the user's query.\n"
        "📦 When returning the result to the user, **always format the output strictly as JSON** using the structure below:\n\n"
        "{\n"
        '  "query": string,\n'
        '  "video_id": string,\n'
        '  "video_url": string,\n'
        '  "title": string,\n'
        '  "description": string,\n'
        '  "duration_seconds": integer,\n'
        '  "published_time": string,\n'
        '  "views": integer,\n'
        '  "thumbnail": string,\n'
        '  "channel": {\n'
        '    "title": string,\n'
        '    "channel_id": string,\n'
        '    "avatar": string,\n'
        '    "channel_url": string\n'
        "  }\n"
        "}\n\n"
        "❗️Do not include any commentary or explanation outside the JSON. Only return valid JSON."
    ),
    name="youtube_assistant"
)


In [ ]:
response = youtube_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="Find a YouTube video that includes the following: - Types: meditation, breathing exercises, motivational - Keywords: stress management, mindfulness, sleep improvement - Mood: calm - Duration: medium (e.g., 0-10min=short, 10-30min=medium, 30min+=long)")
            ]
    }
    )
response["messages"]

[HumanMessage(content='Find a YouTube video that includes the following: - Types: meditation, breathing exercises, motivational - Keywords: stress management, mindfulness, sleep improvement - Mood: calm - Duration: medium (e.g., 0-10min=short, 10-30min=medium, 30min+=long)', additional_kwargs={}, response_metadata={}, id='7dc6daf6-df78-4fac-b207-b6398359a17c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YNU88aPYDDfpCa4VesNTLTv0', 'function': {'arguments': '{"query":"Types: meditation, breathing exercises, motivational; Keywords: stress management, mindfulness, sleep improvement; Mood: calm; Duration: medium"}', 'name': 'search_youtube_video'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 386, 'total_tokens': 427, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens

In [ ]:
response["messages"][-1].content

'{\n  "query": "Types: meditation, breathing exercises, motivational; Keywords: stress management, mindfulness, sleep improvement; Mood: calm; Duration: medium",\n  "video_id": "0E3cbfJnRBE",\n  "video_url": "https://www.youtube.com/watch?v=0E3cbfJnRBE",\n  "title": "Guided 4-7-8 Breathing Technique #relaxing #breathingexercise #meditation",\n  "description": "Enjoy this 4-7-8 breathing technique guided by visual and sounds. 4-7-8 breathing is a breathing technique that can help to ...",\n  "duration_seconds": 20,\n  "published_time": "8 months ago",\n  "views": 40695,\n  "thumbnail": "https://i.ytimg.com/vi/0E3cbfJnRBE/hq720_2.jpg?sqp=-oaymwE2COgCEMoBSFXyq4qpAygIARUAAIhCGABwAcABBvABAfgBtgiAAoAPigIMCAAQARg1IFQofzAP&rs=AOn4CLCNMKdPmZQcpz9pQBzumv81zUIdfA",\n  "channel": {\n    "title": "Headfulness - Luke Horton",\n    "channel_id": "UCAc9_Ug_adJvcKJhAIUqhIQ",\n    "avatar": "https://yt3.ggpht.com/K3EheypHS6P27o67_NVJArpZEDw9a-BK2j5LZ-ayVLHIGHU6sdoTPo2tNNEUHGaghDOjU4qmIck=s68-c-k-c0x00ff

# Spotify Agent

In [ ]:
from langchain.tools import tool
from typing import List
import requests

# Optional: Use a dataclass to format results more clearly
def get_spotify_token(client_id: str, client_secret: str) -> str:
    """
    Fetches a bearer token from Spotify using client credentials.
    """
    url = 'https://accounts.spotify.com/api/token'
    headers = {'Content-Type': 'application/x-www-form-urlencoded'}
    data = {'grant_type': 'client_credentials'}
    response = requests.post(url, headers=headers, data=data, auth=(client_id, client_secret))
    response.raise_for_status()
    return response.json()['access_token']

import requests

@tool
def search_spotify_playlists(query: str) -> dict:
    """
    Search for Spotify playlists using a keyword (e.g., "sleep", "meditation").

    Returns a JSON object with the top 5 playlists including name, description, image, link, owner, and track count.
    """
    client_id = "c62b5f2b76734a77a11c04213b08f446"
    client_secret = "4de968e43fbc45d0a79d118cc25cfcfb"

    try:
        token = get_spotify_token(client_id, client_secret)

        headers = {
            'Authorization': f'Bearer {token}'
        }
        params = {
            'q': query,
            'type': 'playlist',
            'limit': 5
        }
        url = 'https://api.spotify.com/v1/search'
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()

        playlists = response.json().get('playlists', {}).get('items', [])

        if not playlists:
            return {"message": "No playlists found for your query."}

        result = {
            "query": query,
            "total": len(playlists),
            "playlists": []
        }

        for pl in playlists:
            if pl is None:
                continue

            result["playlists"].append({
                "name": pl.get('name'),
                "description": pl.get('description'),
                "external_url": pl.get('external_urls', {}).get('spotify'),
                "image": pl.get('images', [{}])[0].get('url'),
                "owner": {
                    "name": pl.get('owner', {}).get('display_name'),
                    "url": pl.get('owner', {}).get('external_urls', {}).get('spotify')
                },
                "tracks": {
                    "url": pl.get('tracks', {}).get('href'),
                    "total": pl.get('tracks', {}).get('total')
                }
            })

        return result

    except Exception as e:
        return {"error": str(e)}

In [ ]:
spotify_assistant = create_react_agent(
    model=model,
    tools=[search_spotify_playlists],
    prompt=(
        "You are a helpful Spotify assistant. Your job is to help users discover Spotify playlists "
        "based on their search queries using the tool provided.\n\n"
        "🧠 Think step-by-step, and always call the `search_spotify_playlists` tool with the user query.\n"
        "📦 When returning results to the user, format the final output in **JSON** using this structure:\n\n"
        "{\n"
        '  "query": string,\n'
        '  "total": integer,\n'
        '  "playlists": [\n'
        "    {\n"
        '      "name": string,\n'
        '      "description": string,\n'
        '      "external_url": string,\n'
        '      "image": string,\n'
        '      "owner": {"name": string, "url": string},\n'
        '      "tracks": {"url": string, "total": integer}\n'
        "    },\n"
        "    ...\n"
        "  ]\n"
        "}\n\n"
        "Only return JSON. Do not add any extra commentary or text outside of the JSON structure."
    ),
    name="spotify_assistant"
)

# Mood Analyzer

In [ ]:
import chromadb

# Singleton ChromaDB client
chroma_client = chromadb.Client()

def get_collection(name: str):
    return chroma_client.get_or_create_collection(name)

In [ ]:
import re
import json

def extract_json_from_text(text: str) -> dict:
    """
    Extract the first complete JSON object from the given text.
    """
    try:
        json_match = re.search(r'({.*})', text, re.DOTALL)
        if not json_match:
            return {"error": "No JSON object found in response."}
        json_str = json_match.group(1)
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        return {"error": f"Failed to parse JSON: {e}"}

In [ ]:
import os
from langchain_openai import OpenAI
from langchain_core.prompts import PromptTemplate


class MoodAnalyzerAgent:
    def __init__(self):
        self.llm = OpenAI(openai_api_key=key, model="gpt-4o-mini", max_tokens=4096)
        self.prompt = PromptTemplate(
            template="""
            You are MindFuel's AI mood analysis expert. Analyze the user's input text and provide a comprehensive mood assessment in JSON format.
            `CONTEXT:
            {context}
            ANALYSIS REQUIREMENTS:
            1. Detect primary mood and emotional state
            2. Assess confidence level and intensity
            3. Identify key themes, triggers, and emotions
            4. Provide actionable suggestions
            5. Determine content recommendation preferences
            6. Assess any risk factors or concerns

            USER INPUT: "{input}"

            RESPONSE FORMAT (JSON):
            {{
            "analysis": {{
                "primaryMood": "string", // Main detected mood (Happy, Sad, Anxious, Frustrated, Calm, Excited, Depressed, Angry, Neutral, etc.)
                "moodCategory": "string", // positive, negative, neutral
                "confidence": number, // 0-100 confidence score
                "intensity": number, // 1-10 intensity level
                "emotions": [
                {{
                    "emotion": "string",
                    "score": number // 0-100
                }}
                ],
                "sentiment": {{
                "polarity": number, // -1 (very negative) to 1 (very positive)
                "subjectivity": number // 0 (objective) to 1 (subjective)
                }}
            }},
            "insights": {{
                "summary": "string", // 2-3 sentence summary of their emotional state
                "keyThemes": ["string"], // Main themes identified (work, relationships, health, etc.)
                "triggers": ["string"], // Potential triggers or stressors mentioned
                "strengths": ["string"], // Positive aspects or coping mechanisms noted
                "concerns": ["string"] // Areas that might need attention
            }},
            "recommendations": {{
                "immediate": ["string"], // 3-4 immediate actionable suggestions
                "content": {{
                "youtube": {{
                    "types": ["string"], // meditation, breathing exercises, motivational, educational, etc.
                    "keywords": ["string"], // search keywords for relevant videos
                    "duration": "string", // short (0-10min), medium (10-30min), long (30min+)
                    "mood": "string" // target mood for content
                }},
                "articles": {{
                    "topics": ["string"], // psychology, self-help, mindfulness, etc.
                    "difficulty": "string", // beginner, intermediate, advanced
                    "focus": ["string"] // coping strategies, understanding emotions, etc.
                }},
                "spotify": {{
                    "genres": ["string"], // ambient, classical, nature sounds, etc.
                    "energy": number, // 0-1 (0=calm, 1=energetic)
                    "valence": number, // 0-1 (0=sad, 1=happy)
                    "mood": "string" // relaxing, uplifting, focus, etc.
                }},
                "meditation": {{
                    "types": ["string"], // breathing, body scan, loving-kindness, etc.
                    "duration": number, // recommended minutes
                    "difficulty": "string" // beginner, intermediate, advanced
                }}
                }}
            }},
            "followUp": {{
                "questions": ["string"], // 2-3 follow-up questions to better understand their state
                "checkIn": "string", // suggested timeframe for next check-in (hours, days)
                "goals": ["string"] // suggested short-term goals based on their input
            }},
            "riskAssessment": {{
                "level": "string", // low, medium, high
                "indicators": ["string"], // specific risk indicators if any
                "recommendations": ["string"], // professional help suggestions if needed
                "urgency": "string" // none, monitor, immediate
            }},
            "metadata": {{
                "wordCount": number,
                "complexity": "string", // simple, moderate, complex
                "timeOfDay": "string", // morning, afternoon, evening, night (if mentioned)
                "context": ["string"] // work, home, social, health, etc.
            }}
            }}

            GUIDELINES:
            - Be empathetic and non-judgmental
            - Focus on actionable insights
            - Consider cultural sensitivity
            - If serious mental health concerns are detected, prioritize professional help recommendations
            - Tailor content suggestions to the specific mood and needs identified
            - Use clear, supportive language
            - Avoid medical diagnosis or treatment advice
            - Encourage professional help when appropriate

            Analyze the user input and give response in JSON format following the above RESPONSE FORMAT.
            MAKE SURE TO COMPLETE THE ENTIRE JSON STRUCTURE INCLUDING ALL SECTIONS.
            DO NOT TRUNCATE THE RESPONSE.

            IMPORTANT: Your response MUST be a complete JSON object with all sections filled out.
            The response MUST end with the closing brackets for all opened sections.
            DO NOT stop generating until the JSON is complete.`
            """,
            input_variables=["context", "input"]
        )
        self.chain = self.prompt | self.llm
        self.collection = get_collection("mood_analyzer")


    def run(self, user_input, user_id=None, context=None):
        try:
            context_str = """
                - MindFuel is a mental wellness app that helps users track mood, get personalized content recommendations, and improve mental health
                - Users share their thoughts, feelings, and current state
                - Your analysis will be used to provide personalized YouTube videos, articles, Spotify playlists, and meditation recommendations
                - Be empathetic, supportive, and professional in your analysis
            """
            prompt_vars = {
                "context": context_str,
                "input": user_input,
            }

            import re, json

            result = self.chain.invoke(prompt_vars)

            print("-----RAW LLM OUTPUT-----")
            print(result)
            print("------------------------")

            try:
                # Extract the largest JSON block using greedy regex from first "{" to last "}"
                # json_str = re.search(r"\{[\s\S]*\}", result).group(0)

                # # Load JSON
                # parsed_result = extract_json_from_text(result) if isinstance(result, str) else result
                import json

                # Greedy match to catch JSON object, even if it's embedded in a list or contains extra characters
                try:
                    parsed_result = json.loads(result.strip())
                except json.JSONDecodeError:
                    # fallback: extract just the first JSON object using regex
                    import re
                    json_str = re.search(r"\{[\s\S]*\}", result).group(0)
                    parsed_result = json.loads(json_str)

                # Process YouTube recommendation
                youtube_rec = parsed_result.get("recommendations", {}).get("content", {}).get("youtube")
                if youtube_rec:
                    youtube_video = self.get_youtube_video(youtube_rec)
                    parsed_result["recommendations"]["content"]["youtube"]["video"] = youtube_video

                # Process Spotify recommendation
                spotify_rec = parsed_result.get("recommendations", {}).get("content", {}).get("spotify")
                if spotify_rec:
                    spotify_playlist = self.get_spotify_recommendation(spotify_rec)
                    parsed_result["recommendations"]["content"]["spotify"]["playlist"] = spotify_playlist

                return parsed_result

            except Exception as e:
                print("❌ JSON parsing error:", e)
                return {"error": f"Failed to parse JSON: {str(e)}"}
        except Exception as e:
            print("❌ Run error:", e)
            return {"error": str(e)}


#only youtube
    # def run(self, user_input, user_id=None, context=None):
    #     try:
    #         context_str = """
    #             - MindFuel is a mental wellness app that helps users track mood, get personalized content recommendations, and improve mental health
    #             - Users share their thoughts, feelings, and current state
    #             - Your analysis will be used to provide personalized YouTube videos, articles, Spotify playlists, and meditation recommendations
    #             - Be empathetic, supportive, and professional in your analysis
    #         """
    #         prompt_vars = {
    #             "context": context_str,
    #             "input": user_input,
    #         }

    #         import re, json

    #         result = self.chain.invoke(prompt_vars)

    #         print("-----RAW LLM OUTPUT-----")
    #         print(result)
    #         print("------------------------")

    #         try:
    #             # Extract the largest JSON block using greedy regex from first "{" to last "}"
    #             json_str = re.search(r"\{[\s\S]*\}", result).group(0)

    #             # Load JSON
    #             # parsed_result = json.loads(json_str)
    #             parsed_result = extract_json_from_text(result) if isinstance(result, str) else result

    #             # Extract YouTube recommendation block
    #             youtube_rec = parsed_result.get("recommendations", {}).get("content", {}).get("youtube")

    #             if youtube_rec:
    #                 youtube_video = self.get_youtube_video(youtube_rec)
    #                 parsed_result["recommendations"]["content"]["youtube"]["video"] = youtube_video

    #             return parsed_result

    #         except Exception as e:
    #             print("❌ JSON parsing error:", e)
    #             return {"error": f"Failed to parse JSON: {str(e)}"}
    #     except Exception as e:
    #         print(e)
    #         return {"error": str(e)}



        #     print("-----RAW LLM OUTPUT-----")
        #     print(result)
        #     print("------------------------")

        #     # Parse the LLM response JSON
        #     import json
        #     parsed_result = json.loads(result) if isinstance(result, str) else result

        #     # Extract YouTube recommendation block
        #     youtube_rec = parsed_result.get("recommendations", {}).get("content", {}).get("youtube")

        #     # Fetch YouTube video using the assistant
        #     if youtube_rec:
        #         youtube_video = self.get_youtube_video(youtube_rec)
        #         parsed_result["recommendations"]["content"]["youtube"]["video"] = youtube_video
        #     return parsed_result

        # except Exception as e:
        #     print(e)
        #     return {"error": str(e)}

    # def get_youtube_video(self, youtube_recommendation: dict):
    #   try:
    #       # Construct query string
    #       query_parts = []
    #       if "types" in youtube_recommendation:
    #           query_parts.extend(youtube_recommendation["types"])
    #       if "keywords" in youtube_recommendation:
    #           query_parts.extend(youtube_recommendation["keywords"])
    #       if "mood" in youtube_recommendation:
    #           query_parts.append(youtube_recommendation["mood"])
    #       if "duration" in youtube_recommendation:
    #           query_parts.append(f"{youtube_recommendation['duration']} duration")

    #       search_query = " ".join(query_parts)

    #       # Call YouTube assistant
    #       # response = youtube_assistant.invoke(search_query)
    #       response = youtube_assistant.invoke({"query": search_query})
    #       print("🔎 YouTube search query:", search_query)
    #       print("📦 YouTube agent response:", response)
    #       return response
    #   except Exception as e:
    #       return {"error": str(e)}

    from langchain_core.messages import HumanMessage
    import json

    def get_youtube_video(self, youtube_recommendation: dict):
        try:
            # Step 1: Build structured prompt
            types = youtube_recommendation.get("types", [])
            keywords = youtube_recommendation.get("keywords", [])
            mood = youtube_recommendation.get("mood", "")
            duration = youtube_recommendation.get("duration", "")

            prompt = (
                "Find a YouTube video that includes the following:\n"
                f"- Types: {', '.join(types)}\n"
                f"- Keywords: {', '.join(keywords)}\n"
                f"- Mood: {mood}\n"
                f"- Duration: {duration} (e.g., 0-10min=short, 10-30min=medium, 30min+=long)\n\n"
                "Only return JSON with: title, video_url, duration_seconds, thumbnail, and channel details."
            )
            print("🧠 Prompt to YouTube agent:\n", prompt)

            # Step 2: Call YouTube agent
            response = youtube_assistant.invoke({
                "messages": [HumanMessage(content=prompt)]
            })

            # Step 3: Extract the last message's content
            content = response["messages"][-1].content
            print("📦 Raw response content:\n", content)

            # Step 4: Try parsing the JSON content
            if isinstance(content, dict):
                return content  # already a dict

            if isinstance(content, str):
                try:
                    return json.loads(content)
                except json.JSONDecodeError as e:
                    return {"error": f"Failed to parse JSON: {str(e)}"}

            return {"error": "Unknown content format from YouTube assistant."}

        except Exception as e:
            return {"error": f"Exception occurred: {str(e)}"}

    from langchain_core.messages import HumanMessage
    import json

    def get_spotify_recommendation(self, spotify_recommendation: dict):
        try:
            # Step 1: Build structured prompt
            genres = spotify_recommendation.get("genres", [])
            mood = spotify_recommendation.get("mood", "")
            energy = spotify_recommendation.get("energy", "")
            valence = spotify_recommendation.get("valence", "")

            prompt = (
                "Find a Spotify playlist or track recommendation with the following:\n"
                f"- Genres: {', '.join(genres)}\n"
                f"- Mood: {mood}\n"
                f"- Energy level: {energy} (0=low, 1=high)\n"
                f"- Valence (positivity): {valence} (0=sad, 1=happy)\n\n"
                "Only return JSON with: title, spotify_url, duration_seconds, thumbnail, and artist/playlist details."
            )
            print("🎵 Prompt to Spotify agent:\n", prompt)

            # Step 2: Call the Spotify agent
            response = spotify_assistant.invoke({
                "messages": [HumanMessage(content=prompt)]
            })

            # Step 3: Extract the last message's content
            content = response["messages"][-1].content
            print("📦 Raw content from Spotify assistant:\n", content)

            # Step 4: Parse the JSON content
            if isinstance(content, dict):
                return content  # already a dict

            if isinstance(content, str):
                try:
                    return json.loads(content)
                except json.JSONDecodeError as e:
                    return {"error": f"Failed to parse JSON: {str(e)}"}

            return {"error": "Unknown content format from Spotify assistant."}

        except Exception as e:
            return {"error": f"Exception occurred: {str(e)}"}

    # def get_youtube_video(self, youtube_recommendation: dict):
    #     try:
    #         # Step 1: Build the search query components
    #         types = youtube_recommendation.get("types", [])
    #         keywords = youtube_recommendation.get("keywords", [])
    #         mood = youtube_recommendation.get("mood", "")
    #         duration = youtube_recommendation.get("duration", "")

    #         # Step 2: Build a structured prompt
    #         prompt = (
    #             "Find a YouTube video that includes the following:\n"
    #             f"- Types: {', '.join(types)}\n"
    #             f"- Keywords: {', '.join(keywords)}\n"
    #             f"- Mood: {mood}\n"
    #             f"- Duration: {duration} (e.g., 0-10min=short, 10-30min=medium, 30min+=long)\n\n"
    #             "Only return JSON with: title, video_url, duration_seconds, thumbnail, and channel details."
    #         )
    #         print("🧠 Prompt to YouTube agent:\n", prompt)

    #         # Step 3: Invoke YouTube agent
    #         ai_msg = youtube_assistant.invoke({"query": prompt})

    #         # Step 4: Extract and parse content
    #         if not hasattr(ai_msg, "content"):
    #             return {"error": "No content field in YouTube agent response."}

    #         content = ai_msg.content

    #         # If already a dict, return it
    #         if isinstance(content, dict):
    #             return content

    #         # If content is a string, parse as JSON
    #         if isinstance(content, str):
    #             try:
    #                 return json.loads(content)
    #             except json.JSONDecodeError as e:
    #                 return {"error": f"Failed to parse content as JSON: {str(e)}"}

    #         # If it's an AIMessage or other unexpected type
    #         if hasattr(content, "model_dump_json"):
    #             try:
    #                 return json.loads(content.model_dump_json())
    #             except Exception as e:
    #                 return {"error": f"Failed to convert AIMessage content: {str(e)}"}

    #         return {"error": "Unrecognized content format from YouTube agent."}

    #     except Exception as e:
    #         return {"error": str(e)}

    # def get_youtube_video(self, youtube_recommendation: dict):
    #     try:
    #         # Step 1: Build the search query prompt
    #         types = youtube_recommendation.get("types", [])
    #         keywords = youtube_recommendation.get("keywords", [])
    #         mood = youtube_recommendation.get("mood", "")
    #         duration = youtube_recommendation.get("duration", "")

    #         search_query = (
    #             f"Find a YouTube video that includes the following:\n"
    #             f"- Types: {', '.join(types)}\n"
    #             f"- Keywords: {', '.join(keywords)}\n"
    #             f"- Mood: {mood}\n"
    #             f"- Duration: {duration} (e.g., 0-10min=short, 10-30min=medium, 30min+=long)\n\n"
    #             f"Only return JSON with: title, video_url, duration_seconds, thumbnail, and channel details."
    #         )

    #         print("🧠 Prompt to YouTube agent:\n", search_query)

    #         # Step 2: Call the agent
    #         ai_msg = youtube_assistant.invoke({"query": search_query})

    #         # Step 3: Handle both dict and message responses
    #         if isinstance(ai_msg, dict):
    #             return ai_msg  # Already parsed

    #         if hasattr(ai_msg, "content"):
    #             content = ai_msg.content
    #             print("📦 Raw content from YouTube assistant:", content)

    #             if isinstance(content, dict):
    #                 return content

    #             if isinstance(content, str):
    #                 try:
    #                     return json.loads(content)
    #                 except json.JSONDecodeError as e:
    #                     return {"error": f"Failed to parse content as JSON: {str(e)}"}

    #         return {"error": "Unrecognized content format from YouTube assistant."}

    #     except Exception as e:
    #         return {"error": str(e)}


    # def get_youtube_video(self, youtube_recommendation: dict):
    #   try:
    #       # Step 1: Gather parts from the recommendation dict
    #       query_parts = []
    #       query_parts.extend(youtube_recommendation.get("types", []))  # e.g., meditation
    #       query_parts.extend(youtube_recommendation.get("keywords", []))  # e.g., stress relief
    #       mood = youtube_recommendation.get("mood", "")  # e.g., calm
    #       duration = youtube_recommendation.get("duration", "")  # e.g., medium

    #       # Step 2: Construct a natural language prompt
    #       prompt_template = (
    #           "Find a YouTube video that includes the following:\n"
    #           "- Types: {types}\n"
    #           "- Keywords: {keywords}\n"
    #           "- Mood: {mood}\n"
    #           "- Duration: {duration} (e.g., 0-10min=short, 10-30min=medium, 30min+=long)\n\n"
    #           "Only return JSON with: title, video_url, duration_seconds, thumbnail, and channel details."
    #       )

    #       prompt = prompt_template.format(
    #           types=", ".join(youtube_recommendation.get("types", [])) or "any",
    #           keywords=", ".join(youtube_recommendation.get("keywords", [])) or "any",
    #           mood=mood or "any",
    #           duration=duration or "any"
    #       )

    #       print("🧠 Prompt to YouTube agent:\n", prompt)

    #       # Step 3: Invoke YouTube agent with the prompt
    #       ai_msg = youtube_assistant.invoke({"query": prompt})

    #       # Step 4: Validate and parse response
    #       if not hasattr(ai_msg, "content"):
    #           return {"error": "No content field in YouTube agent response."}

    #       content = ai_msg.content
    #       print("📦 Raw content from YouTube assistant:", content)

    #       if isinstance(content, dict):
    #           return content

    #       if isinstance(content, str):
    #           try:
    #               return json.loads(content)
    #           except json.JSONDecodeError as e:
    #               return {"error": f"Failed to parse content as JSON: {str(e)}"}

    #       return {"error": "Unrecognized content format from YouTube assistant."}

    #   except Exception as e:
    #       return {"error": str(e)}

    #work
    # def get_youtube_video(self, youtube_recommendation: dict):
    #   try:
    #       # Construct query string
    #       query_parts = []
    #       if "types" in youtube_recommendation:
    #           query_parts.extend(youtube_recommendation["types"])
    #       if "keywords" in youtube_recommendation:
    #           query_parts.extend(youtube_recommendation["keywords"])
    #       if "mood" in youtube_recommendation:
    #           query_parts.append(youtube_recommendation["mood"])
    #       if "duration" in youtube_recommendation:
    #           query_parts.append(f"{youtube_recommendation['duration']} duration")

    #       search_query = " ".join(query_parts)
    #       print("🔎 YouTube search query:", search_query)

    #       # Invoke the YouTube assistant
    #       ai_msg = youtube_assistant.invoke({"query": search_query})

    #       # SAFELY extract JSON whether it's a string or a dict
    #       if hasattr(ai_msg, "content"):
    #           content = ai_msg.content
    #           print("📦 Raw content from YouTube assistant:", content)

    #           # Try parsing the content if it's a string
    #           if isinstance(content, str):
    #               return json.loads(content)
    #           elif isinstance(content, dict):
    #               return content
    #           else:
    #               return {"error": "Unrecognized content format from YouTube assistant."}

    #       return {"error": "No content returned from YouTube assistant."}

    #   except Exception as e:
    #       return {"error": str(e)}



    # def get_youtube_video(self, youtube_recommendation: dict):
    #     try:
    #         # Construct query string
    #         query_parts = []
    #         if "types" in youtube_recommendation:
    #             query_parts.extend(youtube_recommendation["types"])
    #         if "keywords" in youtube_recommendation:
    #             query_parts.extend(youtube_recommendation["keywords"])
    #         if "mood" in youtube_recommendation:
    #             query_parts.append(youtube_recommendation["mood"])
    #         if "duration" in youtube_recommendation:
    #             query_parts.append(f"{youtube_recommendation['duration']} duration")

    #         search_query = " ".join(query_parts)

    #         print("🔎 YouTube search query:", search_query)

    #         # Invoke the YouTube agent
    #         ai_msg = youtube_assistant.invoke({"query": search_query})

    #         # Ensure it's a valid AIMessage
    #         if hasattr(ai_msg, "content"):
    #             content = ai_msg.content
    #             print("📦 Raw content from YouTube assistant:", content)

    #             # Parse the JSON inside the content
    #             return json.loads(content)

    #         return {"error": "Invalid response structure from YouTube assistant."}

    #     except Exception as e:
    #         return {"error": str(e)}



# Only Youtube

In [ ]:
import json

agent = MoodAnalyzerAgent()

# Provide sample input
user_input = "Lately, I've been feeling overwhelmed with work and unable to focus. I'm not sleeping well, and I get easily irritated."

# Run the agent
output = agent.run(user_input)

# Print the output nicely
print(json.dumps(output, indent=2))

-----RAW LLM OUTPUT-----



USER INPUT: "Lately, I've been feeling overwhelmed with work and unable to focus. I'm not sleeping well, and I get easily irritated."

{
  "analysis": {
    "primaryMood": "Frustrated",
    "moodCategory": "negative",
    "confidence": 85,
    "intensity": 7,
    "emotions": [
      {
        "emotion": "Overwhelm",
        "score": 90
      },
      {
        "emotion": "Irritation",
        "score": 80
      },
      {
        "emotion": "Anxiety",
        "score": 75
      },
      {
        "emotion": "Fatigue",
        "score": 70
      }
    ],
    "sentiment": {
      "polarity": -0.7,
      "subjectivity": 0.9
    }
  },
  "insights": {
    "summary": "You are currently feeling overwhelmed and frustrated due to work-related stress. Your difficulty in focusing and sleeping well suggests a significant emotional burden. It's important to address these feelings to improve your well-being.",
    "keyThemes": ["work", "sleep", "focus"],
    "triggers": ["w

In [ ]:
import json

agent = MoodAnalyzerAgent()

# Provide sample input
user_input = "Lately, I've been feeling overwhelmed with work and unable to focus. I'm not sleeping well, and I get easily irritated."

# Run the agent
output = agent.run(user_input)

# Print the output nicely
print(json.dumps(output, indent=2))

-----RAW LLM OUTPUT-----



USER INPUT: "Lately, I've been feeling overwhelmed with work and unable to focus. I'm not sleeping well, and I get easily irritated."

RESPONSE:

{
  "analysis": {
    "primaryMood": "Overwhelmed",
    "moodCategory": "negative",
    "confidence": 85,
    "intensity": 8,
    "emotions": [
      {
        "emotion": "Anxiety",
        "score": 80
      },
      {
        "emotion": "Frustration",
        "score": 75
      },
      {
        "emotion": "Irritation",
        "score": 70
      },
      {
        "emotion": "Fatigue",
        "score": 65
      }
    ],
    "sentiment": {
      "polarity": -0.7,
      "subjectivity": 0.9
    }
  },
  "insights": {
    "summary": "The user is currently feeling a high level of overwhelm due to work-related stress, which is leading to difficulties in focus and sleep. Irritability is also noted as a significant emotional response.",
    "keyThemes": ["work", "focus", "sleep", "stress"],
    "triggers": ["workload", "i

# Youtube + Spotify

In [ ]:
import json

agent = MoodAnalyzerAgent()

# Provide sample input
user_input = "Lately, I've been feeling overwhelmed with work and unable to focus. I'm not sleeping well, and I get easily irritated."

# Run the agent
output = agent.run(user_input)

# Print the output nicely
print(json.dumps(output, indent=2))

-----RAW LLM OUTPUT-----



USER INPUT: "Lately, I've been feeling overwhelmed with work and unable to focus. I'm not sleeping well, and I get easily irritated."
            """

            {
                "analysis": {
                    "primaryMood": "Frustrated",
                    "moodCategory": "negative",
                    "confidence": 90,
                    "intensity": 8,
                    "emotions": [
                        {
                            "emotion": "Overwhelmed",
                            "score": 85
                        },
                        {
                            "emotion": "Irritated",
                            "score": 75
                        },
                        {
                            "emotion": "Anxious",
                            "score": 70
                        },
                        {
                            "emotion": "Tired",
                            "score": 80
                       